#### 8.1 Hierarchical Indexing

Hierarchical indexing is an important feature of pandas that enables you to have mul‐
tiple (two or more) index levels on an axis. Somewhat abstractly, it provides a way for
you to work with higher dimensional data in a lower dimensional form. Let’s start
with a simple example; create a Series with a list of lists (or arrays) as the index:

In [1]:
import numpy as np
import pandas as pd

In [9]:
data=pd.Series(np.random.randn(9),index=[['a','a','a','b','b','c','c','d','d'],[1,2,3,1, 3, 1, 2, 2, 3]])
data

a  1   -0.388527
   2    1.736341
   3    1.033445
b  1    0.479834
   3   -0.598030
c  1    1.109700
   2   -1.624678
d  2   -0.684800
   3    2.826194
dtype: float64

In [10]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

######  performing indexing

With a hierarchically indexed object, so-called partial indexing is possible, enabling
you to concisely select subsets of the data:


In [11]:
data['a']

1   -0.388527
2    1.736341
3    1.033445
dtype: float64

In [12]:
data['b':'c']

b  1    0.479834
   3   -0.598030
c  1    1.109700
   2   -1.624678
dtype: float64

In [13]:
data[['b','d']]

b  1    0.479834
   3   -0.598030
d  2   -0.684800
   3    2.826194
dtype: float64

one can also use inner index.

In [19]:
data[:,2]

a    1.736341
c   -1.624678
d   -0.684800
dtype: float64

In [15]:
data

a  1   -0.388527
   2    1.736341
   3    1.033445
b  1    0.479834
   3   -0.598030
c  1    1.109700
   2   -1.624678
d  2   -0.684800
   3    2.826194
dtype: float64

###### stack unstack

Hierarchical indexing plays an important role in reshaping data and group-based
operations like forming a pivot table. For example, you could rearrange the data into
a DataFrame using its unstack method.

In [22]:
data.unstack()

,1,2,3
a,-0.388527,1.736341,1.033445
b,0.479834,NaN,-0.598030
c,1.109700,-1.624678,NaN
d,NaN,-0.684800,2.826194


In [23]:
data.unstack().stack()

a  1   -0.388527
   2    1.736341
   3    1.033445
b  1    0.479834
   3   -0.598030
c  1    1.109700
   2   -1.624678
d  2   -0.684800
   3    2.826194
dtype: float64

###### in data frame

In [24]:
df=pd.DataFrame(np.arange(12).reshape(4,3),index=[['a','a','b','b'],[1,2,1,2]],columns=[['Ohio','Ohio','Colorado'],['Red','Green','Red']])
df

Ohio       Colorado
     Red Green      Red
a 1    0     1        2
  2    3     4        5
b 1    6     7        8
  2    9    10       11

In [25]:
df.index

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [27]:
df.columns

MultiIndex([(    'Ohio',   'Red'),
            (    'Ohio', 'Green'),
            ('Colorado',   'Red')],
           )

In [48]:
df.index.names=['key1','key2']
df.columns.names=['state','color']
df

state     Ohio       Colorado
color      Red Green      Red
key1 key2                    
a    1       0     1        2
     2       3     4        5
b    1       6     7        8
     2       9    10       11

In [49]:
df['Ohio']

color      Red  Green
key1 key2            
a    1       0      1
     2       3      4
b    1       6      7
     2       9     10

In [50]:
df['Ohio','Red']

key1  key2
a     1       0
      2       3
b     1       6
      2       9
Name: (Ohio, Red), dtype: int32

In [53]:
df.loc['a','Ohio']['Red']

key2
1    0
2    3
Name: Red, dtype: int32

In [62]:
(df.loc['a','Ohio']).loc[1,'Red']

0

###### pd.MultiIndex.from_arrays

A MultiIndex can be created by itself and then reused; the columns in the preceding
DataFrame with level names could be created like this:

In [63]:
dt=pd.DataFrame(np.arange(12).reshape(-1,3))
dt

,0,1,2
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11


In [72]:
dt.columns=pd.MultiIndex.from_arrays([['Ohio','Ohio','Colorado'],['Red','Green','Red']],names=('state','colour'))
dt

state  Ohio       Colorado
colour  Red Green      Red
0         0     1        2
1         3     4        5
2         6     7        8
3         9    10       11

#### Reordering and Sorting Levels

In [73]:
df

state     Ohio       Colorado
color      Red Green      Red
key1 key2                    
a    1       0     1        2
     2       3     4        5
b    1       6     7        8
     2       9    10       11

In [76]:
df.swaplevel('key2','key1')

state     Ohio       Colorado
color      Red Green      Red
key2 key1                    
1    a       0     1        2
2    a       3     4        5
1    b       6     7        8
2    b       9    10       11

In [80]:
df.swaplevel("state",'color',axis='columns')

color      Red Green      Red
state     Ohio  Ohio Colorado
key1 key2                    
a    1       0     1        2
     2       3     4        5
b    1       6     7        8
     2       9    10       11

In [81]:
df.swaplevel('state','color',axis='columns')['Red']

state      Ohio  Colorado
key1 key2                
a    1        0         2
     2        3         5
b    1        6         8
     2        9        11

In [83]:
df.sort_index(level=1)

state     Ohio       Colorado
color      Red Green      Red
key1 key2                    
a    1       0     1        2
b    1       6     7        8
a    2       3     4        5
b    2       9    10       11

In [84]:
df.sort_index(axis='columns')

state     Colorado  Ohio    
color          Red Green Red
key1 key2                   
a    1           2     1   0
     2           5     4   3
b    1           8     7   6
     2          11    10   9

In [86]:
df.sort_index(axis='columns',level=1)

state      Ohio Colorado Ohio
color     Green      Red  Red
key1 key2                    
a    1        1        2    0
     2        4        5    3
b    1        7        8    6
     2       10       11    9

##### using both swap level and sort index

In [87]:
df.swaplevel(0,1).sort_index(level=0)

state     Ohio       Colorado
color      Red Green      Red
key2 key1                    
1    a       0     1        2
     b       6     7        8
2    a       3     4        5
     b       9    10       11

##### Summary statistics by levels

In [88]:
df

state     Ohio       Colorado
color      Red Green      Red
key1 key2                    
a    1       0     1        2
     2       3     4        5
b    1       6     7        8
     2       9    10       11

In [90]:
df.sum(level='key2')

C:\Users\shraj\AppData\Local\Temp/ipykernel_16084/3695604200.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df.sum(level='key2')


state Ohio       Colorado
color  Red Green      Red
key2                     
1        6     8       10
2       12    14       16

In [93]:
df.sum(level='color',axis=1)

C:\Users\shraj\AppData\Local\Temp/ipykernel_16084/3758348820.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df.sum(level='color',axis=1)


color      Red  Green
key1 key2            
a    1       2      1
     2       8      4
b    1      14      7
     2      20     10

In [100]:
df.sum(level='key2').sum(axis=1)

C:\Users\shraj\AppData\Local\Temp/ipykernel_16084/3525612726.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df.sum(level='key2').sum(axis=1)


key2
1    24
2    42
dtype: int64

In [94]:
df

state     Ohio       Colorado
color      Red Green      Red
key1 key2                    
a    1       0     1        2
     2       3     4        5
b    1       6     7        8
     2       9    10       11

In [95]:
df.sum(level='state',axis=1)

C:\Users\shraj\AppData\Local\Temp/ipykernel_16084/1258278923.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df.sum(level='state',axis=1)


state      Ohio  Colorado
key1 key2                
a    1        1         2
     2        7         5
b    1       13         8
     2       19        11

In [97]:
df.sum(level='state',axis=1).sum()

C:\Users\shraj\AppData\Local\Temp/ipykernel_16084/1801711964.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df.sum(level='state',axis=1).sum()


state
Ohio        40
Colorado    26
dtype: int64

#### Indexing with a DataFrame’s columns.


In [101]:
df

state     Ohio       Colorado
color      Red Green      Red
key1 key2                    
a    1       0     1        2
     2       3     4        5
b    1       6     7        8
     2       9    10       11

In [105]:
df=pd.DataFrame({'a':range(7),'b':range(7,0,-1),'c':['one','one','one', 'two', 'two','two','two'],'d': [0, 1, 2, 0, 1, 2, 3]})
df

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [106]:
df.set_index(['c','d'])

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [110]:
df.set_index(['c','d'],drop=False)


a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [112]:
a=df.set_index(['c','d'])
a

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [115]:
a.reset_index(inplace=True)

In [116]:
a

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


In [117]:
a.sort_index(axis='columns')

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


#### Combining and merging datasets

#### Database-Style DataFrame Joins
 

Merge or join operations combine datasets by linking rows using one or more keys.
 These operations are central to relational databases (e.g., SQL-based). The merge
 function in pandas is the main entry point for using these algorithms on your data.

In [2]:
import pandas as pd
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df2=pd.DataFrame({'key':['a','c','b'],'data2':range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [3]:
df2

,key,data2
0,a,0
1,c,1
2,b,2


In [4]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,2
1,b,1,2
2,b,6,2
3,a,2,0
4,a,4,0
5,a,5,0
6,c,3,1


Note that I didn’t specify which column to join on. If that information is not speci
fied, merge uses the overlapping column names as the keys. It’s a good practice to
specify explicitly, though

In [5]:
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,2
1,b,1,2
2,b,6,2
3,a,2,0
4,a,4,0
5,a,5,0
6,c,3,1


 If the column names are different in each object, you can specify them separately

In [12]:
df1=df1.rename(columns={'key':'lkey'})

In [16]:
df1

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [15]:
df2=df2.rename(columns={'key':'rkey'})
df2

,rkey,data2
0,a,0
1,c,1
2,b,2


In [18]:
pd.merge(df1,df2,left_on='lkey',right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,2
1,b,1,b,2
2,b,6,b,2
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0
6,c,3,c,1


By default merge performs inner join

In [23]:
df1=pd.DataFrame({'key': ['b', 'c', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df2=pd.DataFrame({'key':['a','b','d'],'data2':range(3)})

In [24]:
df1.merge(df2,on='key')

,key,data1,data2
0,b,0,1
1,b,6,1
2,a,2,0
3,a,4,0
4,a,5,0


d and c are missing not included in merge data

Other possible options are 'left',
 'right', and 'outer'

In [25]:
df1.merge(df2,on='key',how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,6.0,1.0
2,c,1.0,NaN
3,c,3.0,NaN
4,a,2.0,0.0
5,a,4.0,0.0
6,a,5.0,0.0
7,d,NaN,2.0


In [26]:
df1.merge(df2,how='left')

,key,data1,data2
0,b,0,1.0
1,c,1,NaN
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


In [27]:
df1.merge(df2,how='right')

,key,data1,data2
0,a,2.0,0
1,a,4.0,0
2,a,5.0,0
3,b,0.0,1
4,b,6.0,1
5,d,NaN,2


To merge with multiple keys

In [31]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],'key2': ['one', 'two', 'one'],'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],'key2': ['one', 'one', 'one', 'two'],'rval': [4, 5, 6, 7]})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [32]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [30]:
pd.merge(left,right,on=['key1','key2'])

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,bar,one,3,6


In [34]:
pd.merge(left,right,on='key1',how='outer')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


 A last issue to consider in merge operations is the treatment of overlapping column
 names.merge has a suffixes option for specifying strings to append
 to overlapping names in the left and right DataFrame objects

In [35]:
pd.merge(left,right,on='key1',how='outer',suffixes=['_left','_right'])

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [38]:
df1=left.rename(columns={'key2':'lkey'})
df2=right.rename(columns={'key2':'rkey'})
df1

,key1,lkey,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [39]:
df2

,key1,rkey,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [43]:
pd.merge(df1,df2,left_on=['key1','lkey'],right_on=['key1','rkey'],suffixes=['_left','_right'])

,key1,lkey,lval,rkey,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,bar,one,3,one,6


###  Merging on Index

In [45]:

df1=pd.DataFrame({'key':['a','a','b','b','c','c'],'val':range(6)})
df2=pd.DataFrame({'val':range(3)},index=list("abd"))
df1

,key,val
0,a,0
1,a,1
2,b,2
3,b,3
4,c,4
5,c,5


In [46]:
df2

,val
a,0
b,1
d,2


In [47]:
pd.merge(df1,df2,left_on='key',right_index=True)

,key,val_x,val_y
0,a,0,0
1,a,1,0
2,b,2,1
3,b,3,1


In [49]:
pd.merge(df1,df2,left_on='key',right_index=True,how='right',suffixes=['_df1','_df2'])

,key,val_df1,val_df2
0.0,a,0.0,0
1.0,a,1.0,0
2.0,b,2.0,1
3.0,b,3.0,1
NaN,d,NaN,2


In [52]:
import numpy as np
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio','Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),index=[['Nevada', 'Nevada', 'Ohio', 'Ohio','Ohio', 'Ohio'],
                                                           [2001, 2000, 2000, 2000, 2001, 2002]], 
                      columns=['event1', 'event2'])
lefth


,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [53]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [54]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [56]:
pd.merge(lefth,righth,left_on='key1',right_index=True)

ValueError: len(left_on) must equal the number of levels in the index of "right"

In [58]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True,how='left')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN


One can use both left_index=True  and right_index=True to merge using only indexes

### Concatinating Along Axis

###### in numpy array  using numpy concatenate

In [59]:
ar=np.arange(12).reshape(4,3)
ar

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [62]:
np.concatenate([ar,ar],axis=1)

array([[ 0,  1,  2,  0,  1,  2],
       [ 3,  4,  5,  3,  4,  5],
       [ 6,  7,  8,  6,  7,  8],
       [ 9, 10, 11,  9, 10, 11]])

###### Using pd.concat

By default concat works along axis=0, producing another Series. If you pass axis=1,
 the result will instead be a DataFrame (axis=1 is the columns)

In [63]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [64]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [66]:
pd.concat([s1,s2,s3],axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [68]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'a', 'b'])
s3 = pd.Series([5, 6], index=['f', 'c'])
pd.concat([s1,s2,s3])

a    0
b    1
c    2
a    3
b    4
f    5
c    6
dtype: int64